In [2]:
import pandas as pd
import requests
import json
import time


In [3]:
artist_names = ['Jonas Brothers', 'Cardi B', 'Panic! At The Disco', 'Queen', 'P!nk', 'Chris Brown','Maroon 5', 'Elton John', 'Lady Gaga', 'Marshmello']

print(artist_names)

['Jonas Brothers', 'Cardi B', 'Panic! At The Disco', 'Queen', 'P!nk', 'Chris Brown', 'Maroon 5', 'Elton John', 'Lady Gaga', 'Marshmello']


In [8]:
#open dictionary in order to use data frame later.
all_artist_dict = {}

#define the function that will collect the artist data
def songs_of_artist(name):
    start_time = time.time() #Check how long it takes your computer to calculate the information for each artist
    
    name = str(name)
    name_low = name.lower() #make all artist-name letters lowercase
    
    #fix the space in the name
    if ' ' in name:
        name_new = "%20".join(name_low.split())
        url = f'http://api.deezer.com/search/artist/?q={name_new}&tracklist&index=0&limit=1&output=json'
    else:
        url = f'http://api.deezer.com/search/artist/?q={name_low}&tracklist&index=0&limit=1&output=json'

    #open a dictionary for every artist
    artist_dict = {}
    song_list = []
    
    
    #requst the general data of the artist:
    artist_data = requests.get(url)
    artist_data_json = artist_data.json()
    track_list_url = artist_data_json['data'][0]['tracklist'] #requst the songs data of the artist (it is a URL):
    
    all_songs_loaded = False
    while not all_songs_loaded: 
        track_list_data = requests.get(track_list_url)
        track_list_json = track_list_data.json()
        for j in range(len(track_list_json['data'])):
            song = track_list_json['data'][j]['title']
            song_list.append(song)
            artist_dict['Song_list'] = song_list

        if track_list_json.get('next'):
            track_list_url = track_list_json['next']
        else:
            all_songs_loaded = True
            
    artist_dict['Number_of_songs'] = len(song_list)
            
    # add time to artist dictionary
    run_time = time.time() - start_time
    artist_dict['Process_time'] = run_time

    # insert artist_dict into all_artist_dict
    all_artist_dict[name] = artist_dict

    return f"Done for {name}"

In [9]:
now = time.time()
for i in artist_names:
    print(songs_of_artist(i))

print('all done')
total_time = time.time()-now
print(f'{len(artist_names)} artist process took {total_time}')

Done for Jonas Brothers
Done for Cardi B
Done for Panic! At The Disco
Done for Queen
Done for P!nk
Done for Chris Brown
Done for Maroon 5
Done for Elton John
Done for Lady Gaga
Done for Marshmello
all done
10 artist process took 8.940851211547852


In [ ]:
#print(all_artist_dict)

In [10]:
# Create dataframe from dic and make keys, index in dataframe
df = pd.DataFrame.from_dict(all_artist_dict, orient='index')
df.reset_index(inplace = True)
df.rename({'index':'Artist'}, axis=1, inplace=True)
df

,Artist,Song_list,Number_of_songs,Process_time
0,Jonas Brothers,"[Only Human, Sucker, What A Man Gotta Do, X, L...",73,1.311919
1,Cardi B,"[I Like It, Bodak Yellow, Please Me, Money, Ro...",48,1.418578
2,Panic! At The Disco,"[High Hopes, Death of a Bachelor, Into the Unk...",98,0.884125
3,Queen,"[Don't Stop Me Now, Bohemian Rhapsody, Under P...",100,0.731714
4,P!nk,"[What About Us, Just Give Me a Reason, So What...",89,0.768544
5,Chris Brown,"[Go Crazy, Big Slimes, Say You Love Me, No Gui...",99,0.809367
6,Maroon 5,"[Memories, Sugar, Girls Like You, This Love, P...",98,0.726923
7,Elton John,"[Your Song, Rocket Man (I Think It's Going To ...",99,0.841001
8,Lady Gaga,"[Rain On Me, Shallow, Stupid Love, Always Reme...",100,0.687278
9,Marshmello,"[Be Kind, Happier, FRIENDS, Bayen Habeit, Wolv...",95,0.752466


In [7]:
#the mean data collection rate:
mean_time = df['Process_time'].mean()
print(f'{mean_time:2.2f} sec')

0.90 sec
